In [90]:
import pandas as pd
import requests

url = 'https://fantasyfootballcalculator.com/draft/6541965'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers=header)
df = pd.read_html(r.text)
df = df[0]
df = df.iloc[:,1:13]

for i in range(0,len(list(df))):

    df.iloc[:,i] = df.iloc[:,i].str.replace(r" .*","")
    df.iloc[:,i] = df.iloc[:, i].str.replace(r'(.*[A-Z]{0,1})([A-Z][a-z])', r'\1  \2')


In [46]:
df = df.iloc[:,1:13]

In [91]:
df

,1,2,3,4,5,6,7,8,9,10,11,12
,Cardog,UN,Hickey,C,Team 5,Team 6,Computer,spo. money,Blitzers,js,quo soo roo,13 yearold Andy Reid
0,ChristianMc Caffrey,Alvin Kamara,Saquon Barkley,Ezekiel Elliott,LeVeon Bell,James Conner,David Johnson,Davante Adams,DeAndre Hopkins,Julio Jones,Nick Chubb,Joe Mixon
1,Antonio Brown,Pat Mahomes,Mike Evans,Travis Kelce,Odell Beckham,Michael Thomas,Tyreek Hill,JuJuSmith- Schuster,Melvin Gordon,Damien Williams,Todd Gurley,Dalvin Cook
2,Aaron Jones,Marlon Mack,Leonard Fournette,Kerryon Johnson,T.Y. Hilton,Amari Cooper,Devonta Freeman,Derrick Henry,Adam Thielen,Keenan Allen,George Kittle,Josh Jacobs
3,Mike Williams,Phillip Lindsay,A.J. Green,Stefon Diggs,Brandin Cooks,Sony Michel,Julian Edelman,David Montgomery,Chris Carson,Zach Ertz,Mark Ingram,Robert Woods
4,Deshaun Watson,Tyler Lockett,Tevin Coleman,Kenny Golladay,Kenyan Drake,O.J. Howard,Andrew Luck,Cooper Kupp,Aaron Rodgers,Chris Godwin,Calvin Ridley,Alshon Jeffery
5,Allen Robinson,Evan Engram,Hunter Henry,D.J. Moore,Matt Ryan,Tarik Cohen,Tyler Boyd,Baker Mayfield,Lamar Miller,Derrius Guice,Robby Anderson,Dante Pettis
6,Rashaad Penny,Will Fuller,Drew Brees,Miles Sanders,James White,Jarvis Landry,Latavius Murray,Eric Ebron,Christian Kirk,Carson Wentz,Russell Wilson,VanceMc Donald
7,Delanie Walker,MarquezValdes- Scantling,Royce Freeman,Jared Goff,Austin Ekeler,Kyler Murray,Kareem Hunt,Darrell Henderson,Jared Cook,Jordan Howard,Sammy Watkins,Marvin Jones
8,Geronimo Allison,Peyton Barber,Sterling Shepard,LeSeanMc Coy,Chicago Defense,Larry Fitzgerald,David Njoku,Ronald Jones,Curtis Samuel,Adrian Peterson,Carlos Hyde,Cam Newton


In [49]:
rest = df.split(' ', 1)[0]

AttributeError: 'DataFrame' object has no attribute 'split'

In [64]:
import re
string = 'HelloCHARLIE this isBob.'
df.iloc[:,0]  = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', df.iloc[:,0])

TypeError: expected string or bytes-like object

In [61]:
range(0,len(list(df)))

range(0, 12)

In [65]:
df.iloc[:0] = df.iloc[:0].str.replace(r'\b([A-Z]{1,2})([A-Z][a-z])', r'\1 \2')


AttributeError: 'DataFrame' object has no attribute 'str'

In [83]:
df.iloc[1,:].str.replace(r'(.*[A-Z]{0,1})([A-Z][a-z])', r'\1  \2')

1   Cardog                  Pat  Mahomes
2   UN                      Pat  Mahomes
3   Hickey                  Pat  Mahomes
4   C                       Pat  Mahomes
5   Team 5                  Pat  Mahomes
6   Team 6                  Pat  Mahomes
7   Computer                Pat  Mahomes
8   spo. money              Pat  Mahomes
9   Blitzers                Pat  Mahomes
10  js                      Pat  Mahomes
11  quo soo roo             Pat  Mahomes
12  13 yearold Andy Reid    Pat  Mahomes
Name: 1, dtype: object

In [84]:
df

,1,2,3,4,5,6,7,8,9,10,11,12
,Cardog,UN,Hickey,C,Team 5,Team 6,Computer,spo. money,Blitzers,js,quo soo roo,13 yearold Andy Reid
0,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara,AlvinKamara
1,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes,PatMahomes
2,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack,MarlonMack
3,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay,PhillipLindsay
4,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett,TylerLockett
5,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram,EvanEngram
6,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller,WillFuller
7,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling,MarquezValdes-Scantling
8,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber,PeytonBarber
